In [4]:
import pandas as pd
from pandas.io import json

In [5]:
path = "../dataset/aptsimulator_cobaltstrike_2021-06-11T21081492.json"
df = json.read_json(path, lines=True)

ValueError: Expected object or value

In [3]:
df.head()

,SourceName,ProviderGuid,Level,Keywords,Channel,Hostname,TimeCreated,@timestamp,EventID,Message,...,ServiceType,ServiceStartType,ServiceAccount,ClientProcessStartKey,ClientProcessId,ImagePath,StartType,AccountName,param1,param2
0,Microsoft-Windows-Sysmon,{5770385f-c22a-43e0-bf4c-06f5698ffbd9},4,0x8000000000000000,Microsoft-Windows-Sysmon/Operational,WORKSTATION5,2021-06-11T09:07:15.635Z,2021-06-11T09:07:15.635Z,1,Process Create:\r\nRuleName: -\r\nUtcTime: 202...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Microsoft-Windows-Sysmon,{5770385f-c22a-43e0-bf4c-06f5698ffbd9},4,0x8000000000000000,Microsoft-Windows-Sysmon/Operational,WORKSTATION5,2021-06-11T09:07:15.636Z,2021-06-11T09:07:15.636Z,10,Process accessed:\r\nRuleName: -\r\nUtcTime: 2...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Microsoft-Windows-Sysmon,{5770385f-c22a-43e0-bf4c-06f5698ffbd9},4,0x8000000000000000,Microsoft-Windows-Sysmon/Operational,WORKSTATION5,2021-06-11T09:07:15.636Z,2021-06-11T09:07:15.636Z,10,Process accessed:\r\nRuleName: -\r\nUtcTime: 2...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Microsoft-Windows-Sysmon,{5770385f-c22a-43e0-bf4c-06f5698ffbd9},4,0x8000000000000000,Microsoft-Windows-Sysmon/Operational,WORKSTATION5,2021-06-11T09:07:15.636Z,2021-06-11T09:07:15.636Z,7,Image loaded:\r\nRuleName: -\r\nUtcTime: 2021-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Microsoft-Windows-Sysmon,{5770385f-c22a-43e0-bf4c-06f5698ffbd9},4,0x8000000000000000,Microsoft-Windows-Sysmon/Operational,WORKSTATION5,2021-06-11T09:07:15.636Z,2021-06-11T09:07:15.636Z,7,Image loaded:\r\nRuleName: -\r\nUtcTime: 2021-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns

Index(['SourceName', 'ProviderGuid', 'Level', 'Keywords', 'Channel',
       'Hostname', 'TimeCreated', '@timestamp', 'EventID', 'Message',
       ...
       'ServiceType', 'ServiceStartType', 'ServiceAccount',
       'ClientProcessStartKey', 'ClientProcessId', 'ImagePath', 'StartType',
       'AccountName', 'param1', 'param2'],
      dtype='object', length=126)

In [5]:
print(df.columns.tolist())

['SourceName', 'ProviderGuid', 'Level', 'Keywords', 'Channel', 'Hostname', 'TimeCreated', '@timestamp', 'EventID', 'Message', 'Task', 'RuleName', 'UtcTime', 'ProcessGuid', 'ProcessId', 'Image', 'FileVersion', 'Description', 'Product', 'Company', 'OriginalFileName', 'CommandLine', 'CurrentDirectory', 'User', 'LogonGuid', 'LogonId', 'TerminalSessionId', 'IntegrityLevel', 'Hashes', 'ParentProcessGuid', 'ParentProcessId', 'ParentImage', 'ParentCommandLine', 'SourceProcessGUID', 'SourceProcessId', 'SourceThreadId', 'SourceImage', 'TargetProcessGUID', 'TargetProcessId', 'TargetImage', 'GrantedAccess', 'CallTrace', 'ImageLoaded', 'Signed', 'Signature', 'SignatureStatus', 'TargetFilename', 'IsExecutable', 'CreationUtcTime', 'EventType', 'TargetObject', 'Details', 'PipeName', 'Device', 'Protocol', 'Initiated', 'SourceIsIpv6', 'SourceIp', 'SourceHostname', 'SourcePort', 'SourcePortName', 'DestinationIsIpv6', 'DestinationIp', 'DestinationHostname', 'DestinationPort', 'DestinationPortName', 'Proce

#### Notes:

- Work on discretizing, normalizing and think of how to create features from the columns and have questions about the data

#### Findings:

- Trying to find columns similar to LANL Denign AD Logs & PurpleSharp AD Log Playbook for Lateral Movement

'SourceName', <br>
'Hostname', <br>
'TimeCreated', <-- 'EventTime' (LANL) <br>
'DestinationIsIpv6', 'DestinationIp', 'DestinationHostname', 'DestinationPort', 'DestinationPortName'; <-- (Look into these) <br>
'SubjectDomainName', <br>
'EventID', <br>
'EventType', <br>
'LogonId', <br>
'ParentProcessId', <br>
'ParentProcessName', <br>
'ProcessID', <br>
'ProcessName', <br>
'NewProcessId', <br>
'NewProcessName', <br>
'ServiceName', <br>
'SubjectUserSid', <br>
'SubjectUserName', <br>
'SubjectDomainName', <br>
'SubjectLogonId', <br>
'Status' <br>

'AccessReason', 'Task', 'Message', 'Description', 'Details' <-- (Look into these)

In [6]:
df_cobalt = df[['EventID', 'EventType', 'LogonId', 'Hostname', 'SubjectUserName', 'SubjectDomainName', 'SubjectLogonId', 
                'Status', 'SourceName', 'ServiceName', 'DestinationHostname', 'ProcessName', 'ProcessID', 'ParentProcessName',
                'ParentProcessId']]

In [7]:
df_cobalt.head()

,EventID,EventType,LogonId,Hostname,SubjectUserName,SubjectDomainName,SubjectLogonId,Status,SourceName,ServiceName,DestinationHostname,ProcessName,ProcessID,ParentProcessName,ParentProcessId
0,1,NaN,0x3719fb,WORKSTATION5,NaN,NaN,NaN,NaN,Microsoft-Windows-Sysmon,NaN,NaN,NaN,NaN,NaN,5632.0
1,10,NaN,NaN,WORKSTATION5,NaN,NaN,NaN,NaN,Microsoft-Windows-Sysmon,NaN,NaN,NaN,NaN,NaN,NaN
2,10,NaN,NaN,WORKSTATION5,NaN,NaN,NaN,NaN,Microsoft-Windows-Sysmon,NaN,NaN,NaN,NaN,NaN,NaN
3,7,NaN,NaN,WORKSTATION5,NaN,NaN,NaN,NaN,Microsoft-Windows-Sysmon,NaN,NaN,NaN,NaN,NaN,NaN
4,7,NaN,NaN,WORKSTATION5,NaN,NaN,NaN,NaN,Microsoft-Windows-Sysmon,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_cobalt.isnull().all(0)

EventID                False
EventType              False
LogonId                False
Hostname               False
SubjectUserName        False
SubjectDomainName      False
SubjectLogonId         False
Status                 False
SourceName             False
ServiceName            False
DestinationHostname    False
ProcessName            False
ProcessID              False
ParentProcessName      False
ParentProcessId        False
dtype: bool

In [9]:
for col in list(df_cobalt):
    print(col)
    print(df_cobalt[col].unique())

EventID
[   1   10    7    5   26   11   13   12   17    9   18    3 5156 5158
 4689 4688 4690 4658 4656 4663 4703 4670 5140 5145 4697 7045 7009 7000]
EventType
[nan 'SetValue' 'CreateKey' 'CreatePipe' 'DeleteKey' 'ConnectPipe']
LogonId
['0x3719fb' nan '0x3e7']
Hostname
['WORKSTATION5']
SubjectUserName
[nan 'APT-Simulator' 'WORKSTATION5$']
SubjectDomainName
[nan 'WORKSTATION5' 'WORKGROUP']
SubjectLogonId
[nan '0x3719fb' '0x3e7']
Status
[nan '0x0' '0x1' '0x2' '0x41d' '0x426' '0x80' '0x7']
SourceName
['Microsoft-Windows-Sysmon' 'Microsoft-Windows-Security-Auditing'
 'Service Control Manager']
ServiceName
[nan 'tbbd05']
DestinationHostname
[nan '-']
ProcessName
[nan 'C:\\Windows\\System32\\PING.EXE'
 'C:\\Users\\APT-Simulator\\Documents\\APTSimulator-master\\helpers\\7z.exe'
 'C:\\Windows\\System32\\conhost.exe' 'C:\\Windows\\System32\\timeout.exe'
 'C:\\TMP\\CreateNamedPipe.exe' 'C:\\Windows\\System32\\taskkill.exe'
 'C:\\Windows\\System32\\svchost.exe'
 'C:\\Windows\\System32\\wbem\\Wmi

In [10]:
df_cobalt.groupby(by=['ServiceName']).get_group('tbbd05')

,EventID,EventType,LogonId,Hostname,SubjectUserName,SubjectDomainName,SubjectLogonId,Status,SourceName,ServiceName,DestinationHostname,ProcessName,ProcessID,ParentProcessName,ParentProcessId
2500,4697,NaN,NaN,WORKSTATION5,APT-Simulator,WORKSTATION5,0x3719fb,NaN,Microsoft-Windows-Security-Auditing,tbbd05,NaN,NaN,NaN,NaN,5632.0
2608,7045,NaN,NaN,WORKSTATION5,NaN,NaN,NaN,NaN,Service Control Manager,tbbd05,NaN,NaN,NaN,NaN,NaN
